https://gis.stackexchange.com/questions/461143/convert-wrf-netcdf-to-tiff

In [96]:
import os
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
import wrf
import numpy as np

In [97]:
folder = 'wrfout'
for filename in os.listdir(folder):
    # Check if the filename contains a colon
    if '%' in filename or ':' in filename:
        new_filename = filename.replace("%", "-")
        new_filename = new_filename.replace(":", "-")
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

In [98]:
root = 'wrfout'
data = nc.Dataset(root + '/wrfout_d02_2025-03-04_08-3A00-3A00')
dominio = "D02"

### Temperature

In [99]:
t2 =  wrf.getvar(data, "T2", timeidx=0)
t2

<xarray.DataArray 'T2' (south_north: 100, west_east: 100)> Size: 40kB
array([[290.85638, 290.8436 , 290.83182, ..., 290.91913, 290.92114,
        290.9236 ],
       [290.852  , 290.83978, 290.82825, ..., 290.90936, 290.91364,
        290.91656],
       [290.84787, 290.83533, 290.82294, ..., 290.9043 , 290.90683,
        290.90912],
       ...,
       [287.04904, 290.26706, 290.5254 , ..., 290.30658, 290.30673,
        290.30685],
       [289.9962 , 290.3023 , 290.53033, ..., 290.30005, 290.30038,
        290.30017],
       [290.11725, 290.36786, 290.5585 , ..., 290.2935 , 290.29315,
        290.29138]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

### Wind

In [100]:
u10 =  wrf.getvar(data, "U10", timeidx=0)
u10

<xarray.DataArray 'U10' (south_north: 100, west_east: 100)> Size: 40kB
array([[-2.2929072 , -2.1961567 , -2.0917535 , ...,  1.4065951 ,
         1.5322704 ,  1.659376  ],
       [-2.3130286 , -2.2153418 , -2.107607  , ...,  1.3680912 ,
         1.4853711 ,  1.6178479 ],
       [-2.3279839 , -2.2316937 , -2.1254563 , ...,  1.3298949 ,
         1.4607544 ,  1.59749   ],
       ...,
       [ 0.01252607, -0.05387934, -0.10536422, ..., -1.7263062 ,
        -1.6673987 , -1.6009696 ],
       [ 0.01707653, -0.04040723, -0.08126502, ..., -1.8163984 ,
        -1.7589241 , -1.6935815 ],
       [ 0.01841262, -0.02947409, -0.06008059, ..., -1.9029524 ,
        -1.8423055 , -1.7786005 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  U at 10 M
    units:        m s-1
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [101]:
#https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398
#https://mst.nerc.ac.uk/wind_vect_convs.html
#! The wind dirección is in convention of meteorological wind direction, which is the direction wind is coming from.

def wind_speed_direction(u10, v10):
    theta_degree = theta_degree = (180 + 180/np.pi * np.arctan2(u10, v10)) % 360
    theta_radians = np.radians(theta_degree)
    v_mod = np.sqrt(u10**2 + v10**2)
    u_vel = v_mod * np.cos(theta_radians)
    v_vel = v_mod * np.sin(theta_radians)
    return(theta_degree, v_mod, u_vel, v_vel)

In [102]:
u10 = wrf.getvar(data, 'U10', timeidx=0)  
v10 = wrf.getvar(data, 'V10', timeidx=0)

nx = data.dimensions['west_east'].size
ny = data.dimensions['south_north'].size
dt, dx, dy = data.DT, data.DX, data.DY
cen_lat, cen_lon = data.CEN_LAT, data.CEN_LON  # center of the domain
truelat1, truelat2 = data.TRUELAT1, data.TRUELAT2  # true latitudes
pole_lat, pole_lon = data.POLE_LAT, data.POLE_LON  # coordinates of the pole    

print(nx, ny, cen_lat, cen_lon)

100 100 28.080208 -15.561523


In [103]:
theta_degree, v_mod, u_vel, v_vel = wind_speed_direction(u10, v10)

### Wind by height

In [104]:
wspd =  wrf.getvar(data, "wspd", timeidx=0)
z =  wrf.getvar(data, "z", timeidx=0) #model height - [MSL] (mass grid)
hgt =  wrf.getvar(data, "HGT", timeidx=0) #Terrain Height
z_agl = z - hgt #altura sobre el terreno
z_agl

<xarray.DataArray (bottom_top: 44, south_north: 100, west_east: 100)> Size: 2MB
array([[[   25.359447,    25.358612,    25.357838, ...,    25.365866,
            25.366047,    25.36625 ],
        [   25.358868,    25.358221,    25.357557, ...,    25.36616 ,
            25.366268,    25.366253],
        [   25.358248,    25.357603,    25.356894, ...,    25.366165,
            25.366194,    25.366257],
        ...,
        [   25.205414,    25.244095,    25.274437, ...,    25.305117,
            25.304407,    25.303907],
        [   25.212074,    25.247108,    25.274826, ...,    25.304363,
            25.304281,    25.303316],
        [   25.224754,    25.252815,    25.276508, ...,    25.302607,
            25.30266 ,    25.302746]],

       [[   78.56045 ,    78.55779 ,    78.55535 , ...,    78.58749 ,
            78.588104,    78.58883 ],
        [   78.55859 ,    78.55677 ,    78.554214, ...,    78.58886 ,
            78.58932 ,    78.588844],
        [   78.556625,    78.55434 ,    78.55208 , ...,    78.588776,
            78.58893 ,    78.58885 ],
...
        [19037.715   , 19063.293   , 19085.346   , ..., 19132.518   ,
         19132.58    , 19132.447   ],
        [19042.678   , 19066.531   , 19087.256   , ..., 19132.455   ,
         19132.533   , 19132.396   ],
        [19052.191   , 19073.307   , 19091.666   , ..., 19132.24    ,
         19132.291   , 19132.342   ]],

       [[20068.701   , 20068.64    , 20068.584   , ..., 20066.941   ,
         20066.95    , 20066.959   ],
        [20068.596   , 20068.51    , 20068.438   , ..., 20067.049   ,
         20067.068   , 20066.889   ],
        [20068.494   , 20068.39    , 20068.312   , ..., 20066.973   ,
         20066.98    , 20066.822   ],
        ...,
        [19968.508   , 19994.012   , 20016.031   , ..., 20064.066   ,
         20064.137   , 20063.916   ],
        [19973.508   , 19997.293   , 20018.      , ..., 20064.006   ,
         20064.102   , 20063.89    ],
        [19983.062   , 20004.166   , 20022.508   , ..., 20063.734   ,
         20063.803   , 20063.865   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top, south_north, west_east

In [105]:
from wrf import interplevel
wspd_100m = interplevel(wspd, z_agl, 50)
wspd_100m

<xarray.DataArray 'wspd_wdir_interp' (south_north: 100, west_east: 100)> Size: 40kB
array([[9.065835 , 9.167515 , 9.2681055, ..., 7.6967683, 7.606853 ,
        7.512447 ],
       [9.148724 , 9.245498 , 9.355138 , ..., 7.713824 , 7.6173935,
        7.51788  ],
       [9.230291 , 9.323122 , 9.42573  , ..., 7.695774 , 7.5914063,
        7.5030813],
       ...,
       [3.0777476, 3.2916155, 3.570599 , ..., 6.7853246, 6.872337 ,
        6.962861 ],
       [3.0485418, 3.2425106, 3.5077145, ..., 6.7115855, 6.793528 ,
        6.883582 ],
       [3.0576153, 3.2406998, 3.4610755, ..., 6.6349864, 6.7208095,
        6.8086042]], dtype=float32)
Coordinates:
    wspd_wdir  <U4 16B 'wspd'
    XLONG      (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT       (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME      float32 4B 120.0
    Time       datetime64[ns] 8B 2025-03-04T08:00:00
    level      int32 4B 50
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          m s-1
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LambertConformal(stand_lon=-17.017000198364258, moad_cen_...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     None

### Geopotential Height

In [106]:
ph =  wrf.getvar(data, "PH", timeidx=0)
ph

<xarray.DataArray 'PH' (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[7.29736519e+00, 7.28094006e+00, 7.26574230e+00, ...,
         7.42326832e+00, 7.42680740e+00, 7.43079424e+00],
        [7.28596878e+00, 7.27326965e+00, 7.26025772e+00, ...,
         7.42903376e+00, 7.43114758e+00, 7.43086767e+00],
        [7.27380943e+00, 7.26118088e+00, 7.24721527e+00, ...,
         7.42912865e+00, 7.42973089e+00, 7.43093586e+00],
...
        [5.92983447e+03, 5.92696338e+03, 5.92587207e+03, ...,
         5.94550732e+03, 5.94623486e+03, 5.94456885e+03],
        [5.93012354e+03, 5.92735352e+03, 5.92649121e+03, ...,
         5.94491650e+03, 5.94584619e+03, 5.94419678e+03],
        [5.93041406e+03, 5.92959961e+03, 5.92879785e+03, ...,
         5.94259473e+03, 5.94322119e+03, 5.94384375e+03]],

       [[6.23505811e+03, 6.23454541e+03, 6.23407324e+03, ...,
         6.21661182e+03, 6.21651562e+03, 6.21643604e+03],
        [6.23396729e+03, 6.23332520e+03, 6.23271777e+03, ...,
         6.21802783e+03, 6.21801855e+03, 6.21570312e+03],
        [6.23288135e+03, 6.23207227e+03, 6.23140381e+03, ...,
         6.21728564e+03, 6.21710645e+03, 6.21500098e+03],
        ...,
        [6.17612549e+03, 6.17275439e+03, 6.17156201e+03, ...,
         6.19632471e+03, 6.19698828e+03, 6.19430225e+03],
        [6.17671094e+03, 6.17347705e+03, 6.17258447e+03, ...,
         6.19572217e+03, 6.19667139e+03, 6.19417676e+03],
        [6.17730371e+03, 6.17648389e+03, 6.17566943e+03, ...,
         6.19272363e+03, 6.19340283e+03, 6.19406641e+03]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  perturbation geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [107]:
phb =  wrf.getvar(data, "PHB", timeidx=0)
phb

<xarray.DataArray 'PHB' (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        ...,
        [   919.18726,    665.92566,    448.87518, ...,      0.     ,
              0.     ,      0.     ],
        [   870.5609 ,    634.3602 ,    430.4086 , ...,      0.     ,
              0.     ,      0.     ],
        [   777.322  ,    569.5402 ,    388.8537 , ...,      0.     ,
              0.     ,      0.     ]],

       [[   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
        [   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
        [   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
...
        [186297.61   , 186297.67   , 186297.77   , ..., 186297.92   ,
         186297.92   , 186297.92   ],
        [186297.61   , 186297.73   , 186297.8    , ..., 186297.92   ,
         186297.92   , 186297.92   ],
        [186297.67   , 186297.7    , 186297.77   , ..., 186297.92   ,
         186297.92   , 186297.92   ]],

       [[195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        ...,
        [195216.94   , 195217.     , 195217.1    , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195216.94   , 195217.06   , 195217.12   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.     , 195217.03   , 195217.1    , ..., 195217.25   ,
         195217.25   , 195217.25   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  base-state geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

In [108]:
# La altura geopotencial es una medida que toma en cuenta la variación de la gravedad con la altura y la latitud
# Se puede considerar igual que la altura física para la troposfera
z_gph = (ph + phb) / 9.81
z_gph

<xarray.DataArray (bottom_top_stag: 45, south_north: 100, west_east: 100)> Size: 2MB
array([[[    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        ...,
        [   93.699005,    67.882324,    45.756897, ...,     0.      ,
             0.      ,     0.      ],
        [   88.74219 ,    64.66465 ,    43.874474, ...,     0.      ,
             0.      ,     0.      ],
        [   79.23772 ,    58.057106,    39.6385  , ...,     0.      ,
             0.      ,     0.      ]],

       [[   50.718895,    50.717224,    50.715675, ...,    50.73173 ,
            50.732094,    50.7325  ],
        [   50.717735,    50.716442,    50.715115, ...,    50.73232 ,
            50.732536,    50.732506],
        [   50.716496,    50.715206,    50.713787, ...,    50.73233 ,
            50.732388,    50.732513],
...
        [19595.049   , 19594.764   , 19594.662   , ..., 19596.678   ,
         19596.754   , 19596.582   ],
        [19595.08    , 19594.81    , 19594.727   , ..., 19596.62    ,
         19596.713   , 19596.547   ],
        [19595.115   , 19595.035   , 19594.959   , ..., 19596.383   ,
         19596.445   , 19596.51    ]],

       [[20535.402   , 20535.35    , 20535.303   , ..., 20533.521   ,
         20533.512   , 20533.504   ],
        [20535.291   , 20535.227   , 20535.164   , ..., 20533.666   ,
         20533.666   , 20533.43    ],
        [20535.18    , 20535.1     , 20535.031   , ..., 20533.59    ,
         20533.572   , 20533.357   ],
        ...,
        [20529.363   , 20529.025   , 20528.914   , ..., 20531.455   ,
         20531.521   , 20531.248   ],
        [20529.424   , 20529.107   , 20529.021   , ..., 20531.393   ,
         20531.49    , 20531.234   ],
        [20529.488   , 20529.41    , 20529.334   , ..., 20531.086   ,
         20531.156   , 20531.225   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: bottom_top_stag, south_north, west_east

### Height

In [109]:
# Model Height for Mass Grid (MSL, Meal Sea Level)
# Es la altura física real desde un punto de referencia (generalmente el nivel del mar). No corrige por la variación de la gravedad
hgt =  wrf.getvar(data, "HGT", timeidx=0)
hgt

<xarray.DataArray 'HGT' (south_north: 100, west_east: 100)> Size: 40kB
array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [93.699005, 67.882324, 45.756897, ...,  0.      ,  0.      ,
         0.      ],
       [88.74219 , 64.66465 , 43.874474, ...,  0.      ,  0.      ,
         0.      ],
       [79.23772 , 58.057106, 39.6385  , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 40kB -16.13 -16.12 ... -14.99
    XLAT     (south_north, west_east) float32 40kB 27.59 27.59 ... 28.57 28.57
    XTIME    float32 4B 120.0
    Time     datetime64[ns] 8B 2025-03-04T08:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Terrain Height
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-17.017000198364258, moad_cen_la...

#### Aspect (orientación) and slope (pendiente)

In [110]:
grad_y, grad_x = np.gradient(hgt, dy, dx)

slope = np.arctan(np.sqrt(grad_x**2 + grad_y**2))
slope_deg = np.degrees(slope)

# Aspecto (orientación del terreno)
aspect = np.arctan2(-grad_y, grad_x)
aspect_deg = np.degrees(aspect)
aspect_deg = (aspect_deg + 360) % 360

In [111]:
xmin = float(np.min(t2.XLONG).values)
xmax = float(np.max(t2.XLONG).values)
ymin = float(np.min(t2.XLAT).values)
ymax = float(np.max(t2.XLAT).values)


print( xmin, xmax, ymin, ymax)

-16.130859375 -14.98687744140625 27.574443817138672 28.583740234375


In [112]:
#Extracting coordinates
latitudes = data.variables["XLAT"][0,:,:]
longitudes = data.variables["XLONG"][0,:,:]

# take a single time step
time0_T2 = data.variables["T2"][0,:,:]

# # take a signgle wind step
# time0_U10 = data.variables["U10"][0,:,:]
# time0_V10 = data.variables["V10"][0,:,:]

# defining lists for the output dataset
lat, lon, temp, wind_speed_10m, wind_direction_10m, height, slope, aspect, nc = [], [], [], [], [], [], [], [], []

In [113]:
# counter variable for cell indexing
count = 1
for i in range(latitudes.shape[0]):
    for j in range(latitudes.shape[1]):        
        latitude = latitudes[i, j]
        longitude = longitudes[i, j]
        if ymin <= latitude <= ymax and xmin <= longitude <= xmax:               
            lat.append(latitude)
            lon.append(longitude)
            # convert temperature from Kelvin to Celsius
            temp.append(time0_T2[i, j])
            wind_speed_10m.append(v_mod[i, j])
            wind_direction_10m.append(theta_degree[i, j])
            height.append(hgt[i, j])
            slope.append(slope_deg[i, j])
            aspect.append(aspect_deg[i, j]) 
            nc.append(count)
            count = count + 1

In [114]:
# converting to numpy column array
A = np.transpose(np.array([nc,lon, lat, temp,wind_speed_10m, wind_direction_10m, height, slope, aspect]))
A

array([[ 1.00000000e+00, -1.61308594e+01,  2.75864868e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.00000000e+00, -1.61195068e+01,  2.75864029e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.00000000e+00, -1.61081238e+01,  2.75863342e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 9.99800000e+03, -1.50097961e+01,  2.85719261e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.99900000e+03, -1.49983521e+01,  2.85717545e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+04, -1.49868774e+01,  2.85715828e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [115]:
# writing data to csv
np.savetxt('time0_T2.csv',A,delimiter=",",header="cell_number, LON, LAT, T2_degC,WIND_SPEED_10m, WIND_DIRECTION_10m, HEIGHT, SLOPE, ASPECT")

In [116]:
# # x and y resolutions
deltax = (np.max(A[:,1]) - np.min(A[:,1]))/ (nx - 1)
deltay = (np.max(A[:,2]) - np.min(A[:,2]))/ (ny - 1)

print(deltax, deltay)

0.011555373066603536 0.010194913305417455


In [117]:
# # up left corner
minx = np.min(A[:,1])
maxy = np.max(A[:,2])

print(minx, maxy)

-16.130859375 28.583740234375


In [118]:
# set the transformation for rasterio. Note that the shift operated is in order to
# make the lat,lon points to be the center of each grid cell
transform = from_origin(minx-0.5*deltax,maxy+0.5*deltay, deltax, deltay)
transform

Affine(0.011555373066603536, 0.0, -16.136637061533303,
       0.0, -0.010194913305417455, 28.58883769102771)

In [119]:
t2 = A[:,3]
data_band = np.flipud(t2.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Temperature_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=t2.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [120]:
wind_speed = A[:,4]
data_band = np.flipud(wind_speed.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Wind_Speed_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_speed.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [121]:
wind_direction = A[:,5]
data_band = np.flipud(wind_direction.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Wind_Direction_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_direction.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [122]:
height = A[:,6]
data_band = np.flipud(height.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Height_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=height.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [123]:
slope = A[:,7]
data_band = np.flipud(slope.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Slope_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=slope.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [124]:
Aspect = A[:,8]
data_band = np.flipud(Aspect.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'Aspect_{dominio}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=Aspect.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))